In [46]:
import pandas as pd
from scipy import stats

df = pd.read_csv('../data/stmt.csv')

# Convert to numeric
df['Amount'] = pd.to_numeric(df['Amount'].str.replace(',', '', regex=False))
df['Running Bal.'] = pd.to_numeric(df['Running Bal.'].str.replace(',', '', regex=False))

# Convert to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

print(df.head())

print(df.dtypes)


        Date                                        Description   Amount  \
0 2024-07-05  DFAS-CLEVELAND DES:FED SALARY ID:XXXXX2985 IND...  2605.67   
1 2024-07-08  Online Banking transfer from SAV 2175 Confirma...   300.00   
2 2024-07-08           Bank of America Credit Card Bill Payment  -403.93   
3 2024-07-12  PANINI KABOB GRILL BAKE 07/10 PURCHASE BAKERSF...   -26.17   
4 2024-07-15  INDIAN WELLS VALLEY WAT 07/11 PURCHASE 760-375...   -62.92   

   Running Bal.  
0       2605.67  
1       2905.67  
2       2501.74  
3       2475.57  
4       2412.65  
Date            datetime64[ns]
Description             object
Amount                 float64
Running Bal.           float64
dtype: object


In [47]:
# Caclculate descriptive statistics
print(df['Amount'].describe(include='all'))

# Skewness and Kurtosis
print(stats.skew(df['Amount'].dropna()))
print(stats.kurtosis(df['Amount'].dropna(), fisher=False)) # Use fisher=True for excess kurtosis


count     682.000000
mean        4.250117
std       705.771333
min     -5721.000000
25%      -114.922500
50%       -40.000000
75%       -14.000000
max      5164.000000
Name: Amount, dtype: float64
2.3330335598872125
22.437392985905056


In [48]:
# Downsample dataset to Monthly Cashflow
# Set date as index
df.set_index('Date', inplace=True)

# Resample by month and sum the cashflow
monthly_df = df.resample('M').sum()
monthly_df = monthly_df.drop(['Description'], axis=1)
print(monthly_df)

             Amount  Running Bal.
Date                             
2024-07-31  2394.02      63796.88
2024-08-31  3226.92      93133.27
2024-09-30 -3414.02      99083.98
2024-10-31  -592.48      57902.11
2024-11-30   926.08      48153.44
2024-12-31  -298.33      96274.57
2025-01-31  2486.45     138170.51
2025-02-28 -1082.11     246584.64
2025-03-31  1386.30     382585.95
2025-04-30 -1181.62     341916.79
2025-05-31 -2162.82     271474.11
2025-06-30   -99.73     120897.51
2025-07-31  1309.92       9192.98


C:\Users\KylePC\AppData\Local\Temp\ipykernel_9976\1719393271.py:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_df = df.resample('M').sum()
